In [21]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LayerNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools

In [2]:
data = pd.read_csv('hand_landmarks_dataset.csv')

In [3]:
data['label'].value_counts()

label
Doing other things               38184
Swiping Down                     16650
Thumb Up                         16502
Swiping Left                     16243
Sliding Two Fingers Left         16169
Pulling Hand In                  16169
Zooming Out With Two Fingers     16095
Swiping Up                       16095
Sliding Two Fingers Down         15873
Rolling Hand Forward             15873
Zooming In With Full Hand        15799
Zooming In With Two Fingers      15725
Pushing Two Fingers Away         15725
Pushing Hand Away                15614
Zooming Out With Full Hand       15577
Thumb Down                       15577
Pulling Two Fingers In           15466
Sliding Two Fingers Right        15355
Shaking Hand                     15318
Drumming Fingers                 15244
Stop Sign                        14948
No gesture                       14763
Swiping Right                    14689
Sliding Two Fingers Up           14615
Rolling Hand Backward            13986
Turning Hand Clockw

In [4]:
data.isnull().sum()

label            0
label_id         0
path             0
landmark_0_x     0
landmark_0_y     0
                ..
landmark_19_y    0
landmark_19_z    0
landmark_20_x    0
landmark_20_y    0
landmark_20_z    0
Length: 66, dtype: int64

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434269 entries, 0 to 434268
Data columns (total 66 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   label          434269 non-null  object 
 1   label_id       434269 non-null  int64  
 2   path           434269 non-null  int64  
 3   landmark_0_x   434269 non-null  float64
 4   landmark_0_y   434269 non-null  float64
 5   landmark_0_z   434269 non-null  float64
 6   landmark_1_x   434269 non-null  float64
 7   landmark_1_y   434269 non-null  float64
 8   landmark_1_z   434269 non-null  float64
 9   landmark_2_x   434269 non-null  float64
 10  landmark_2_y   434269 non-null  float64
 11  landmark_2_z   434269 non-null  float64
 12  landmark_3_x   434269 non-null  float64
 13  landmark_3_y   434269 non-null  float64
 14  landmark_3_z   434269 non-null  float64
 15  landmark_4_x   434269 non-null  float64
 16  landmark_4_y   434269 non-null  float64
 17  landmark_4_z   434269 non-nul

In [6]:
data.groupby('label_id').label.value_counts()

label_id  label                        
0         Doing other things               38184
1         Drumming Fingers                 15244
2         No gesture                       14763
3         Pulling Hand In                  16169
4         Pulling Two Fingers In           15466
5         Pushing Hand Away                15614
6         Pushing Two Fingers Away         15725
7         Rolling Hand Backward            13986
8         Rolling Hand Forward             15873
9         Shaking Hand                     15318
10        Sliding Two Fingers Down         15873
11        Sliding Two Fingers Left         16169
12        Sliding Two Fingers Right        15355
13        Sliding Two Fingers Up           14615
14        Stop Sign                        14948
15        Swiping Down                     16650
16        Swiping Left                     16243
17        Swiping Right                    14689
18        Swiping Up                       16095
19        Thumb Down         

In [7]:
X, y = data.drop(columns=['label','label_id', 'path']), data['label_id']

In [8]:
def super_frame(data, sequence_length = 37, num_features = 63):
    raw_data = data.to_numpy()
    n_sequences = len(raw_data) // sequence_length
    raw_data = raw_data[:n_sequences * sequence_length]
    X_seq = raw_data.reshape((n_sequences, sequence_length, num_features))

    # "Суперкадр" — среднее по последовательности
    X_mean = np.mean(X_seq, axis=1, keepdims=True)  # (n_sequences, 1, 63)

    # Добавляем к последовательности
    X_augmented = np.concatenate([X_seq, X_mean], axis=1)  # (n_sequences, 38, 63)

    X = pd.DataFrame(X_augmented.reshape(38*n_sequences, 63))
    
    return X


In [9]:
prep_X = super_frame(X)

In [10]:
minmax = MinMaxScaler()
X_scaled = minmax.fit_transform(prep_X)

window_size_x = 38
window_size_y = 37
n_samples = len(X_scaled) // window_size_x
n_samples_y = len(y) // window_size_y

y_seq = np.array(y[:n_samples_y * window_size_y]).reshape(n_samples_y, window_size_y)

y_seq = y_seq[:, 0] 

X_scaled = X_scaled.reshape(n_samples, 38, 63)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_seq, train_size=0.8, random_state=13)

In [11]:
y_train = to_categorical(y_train, num_classes=27)
y_test = to_categorical(y_test, num_classes=27)

In [12]:
X_train

array([[[0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        ...,
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862]],

       [[0.52077936, 0.47730463, 0.52501584, ..., 0.559212  ,
         0.48411334, 0.79895805],
        [0.52071049, 0.4760113 , 0.52500516, ..., 0.5590961 ,
         0.47315966, 0.79582027],
        [0.08757717, 0.03034138, 0.48643002, ..., 0.14308749,
         0.25328326, 0.80484862],
        ...,
        [0.55241313, 0.50270637, 0.53885878, ..., 0.44449754,
         0.46413642, 0.74665566],
        [0.5

In [23]:
best_acc = 0
best_model = 0

param_grid = {
    'units': [32, 64, 96],
    'dense_units': [8, 16, 32],
    'dropout': [0.2, 0.3],
    'recurrent_dropout': [0.0, 0.2],
    'lr': [1e-3, 5e-4]
}

# Генерация всех комбинаций
all_combinations = list(itertools.product(
    param_grid['units'],
    param_grid['dense_units'],
    param_grid['dropout'],
    param_grid['recurrent_dropout'],
    param_grid['lr']
))

# Перебор комбинаций
for i, (units, dense_units, dropout_rate, rec_dropout, lr) in enumerate(all_combinations):
    print(f"\nКомбинация {i+1}/{len(all_combinations)}: "
          f"units={units}, dense_units={dense_units}, dropout={dropout_rate}, "
          f"recurrent_dropout={rec_dropout}, lr={lr}")

    model = Sequential([
        Bidirectional(GRU(units, return_sequences=True, input_shape=(38, 63), recurrent_dropout=rec_dropout)),
        LayerNormalization(),
        GRU(units, recurrent_dropout=rec_dropout),
        Dropout(dropout_rate),
        Dense(dense_units, activation='relu'),
        Dense(27, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True
    )

    checkpoint = ModelCheckpoint(
        f"model_{i}_best.keras",
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )

    callbacks = [early_stop, checkpoint]

    history = model.fit(
        X_train, y_train,
        epochs=300,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=callbacks,
        verbose=2
    )

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print(f"Тестовая точность: {test_acc:.4f}")

    with open("log.txt", "a") as f:
        f.write(f"Model {i}: acc={test_acc:.4f}, loss={test_loss:.4f}, "
                f"units={units}, dense={dense_units}, dropout={dropout_rate}, "
                f"rec_dropout={rec_dropout}, lr={lr}\n")

    if test_acc > best_acc:
        best_acc = test_acc
        best_model = i




Комбинация 1/72: units=32, dense_units=8, dropout=0.2, recurrent_dropout=0.0, lr=0.001
Epoch 1/300

Epoch 1: val_loss improved from inf to 3.16301, saving model to model_0_best.keras
294/294 - 7s - 24ms/step - accuracy: 0.0756 - loss: 3.2347 - val_accuracy: 0.0865 - val_loss: 3.1630
Epoch 2/300

Epoch 2: val_loss improved from 3.16301 to 2.98985, saving model to model_0_best.keras
294/294 - 4s - 15ms/step - accuracy: 0.0977 - loss: 3.1094 - val_accuracy: 0.1154 - val_loss: 2.9899
Epoch 3/300

Epoch 3: val_loss improved from 2.98985 to 2.64314, saving model to model_0_best.keras
294/294 - 5s - 16ms/step - accuracy: 0.1390 - loss: 2.8627 - val_accuracy: 0.2044 - val_loss: 2.6431
Epoch 4/300

Epoch 4: val_loss improved from 2.64314 to 2.36448, saving model to model_0_best.keras
294/294 - 4s - 15ms/step - accuracy: 0.2131 - loss: 2.5575 - val_accuracy: 0.2704 - val_loss: 2.3645
Epoch 5/300

Epoch 5: val_loss improved from 2.36448 to 2.22297, saving model to model_0_best.keras
294/294 - 4s